In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import xlrd
# the different initial condition is different input element
# the input element need to "Normalization"
# Taking off the label and then the output will become a single node
# the furture work is 
"""
1. Normalization the input (OKAY)
2. Set the initial condition to one element input (OKAY)
3. Reducing the input conditio to robust the model!! (OKAY)
4. the hiddent layer only need one layer in the layer!!(OKay)
5. (4 important input+ 1 initialcondition input) one output
"""
#the post-process
#Input the Excel becomne a input
data = xlrd.open_workbook('traindatabase.xlsx')
database= []
for n in range(len(data.sheet_names())): #len(data.sheet_names()) is the number of worksheet 
    print('worksheet {} :'.format(n),end='')
    table = data.sheets()[n]
    for i in range(table.nrows):
        if i != 0: # neglect the first row!!
            #print(table.row_values(i))
            database.append(table.row_values(i))
print('the number of data = ',len(database)) # the data length
print('the element of each data is',len(database[0])) # How many input
#the data is list so the format is database[row][col]
# Split the database to np.array formate(train element,the number of data)
trainX = np.zeros((len(database[0])-1,len(database)))
trainY = np.zeros((1,len(database)))
for i in range(len(database)):
    n=0
    for j in range(1,len(database[0])):
        trainX[n,i]=database[i][j]
        n=n+1
    trainY[0,i] =database[i][0]
# Making the label of the number 100 200 300 400 500
#for i in range(len(database)):
    #if database[i][0] == 100:
        #train_y[:,i]=np.transpose([1,0,0,0,0])
    #if database[i][0] == 200:
        #train_y[:,i]=np.transpose([0,1,0,0,0])
    #if database[i][0] == 300:
        #train_y[:,i]=np.transpose([0,0,1,0,0])
    #if database[i][0] == 400:
        #train_y[:,i]=np.transpose([0,0,0,1,0])
    #if database[i][0] == 500:
        #train_y[:,i]=np.transpose([0,0,0,0,1])
# Changing the formate to suit the traning set [[output 1 element],[input 16 element]]
#train_data = [[[],[]]]*len(database)
#for i in range(0,len(database)):
    #train_data[i][0]=[database[i][0]]
    #train_data[i][1]=database[i][1:len(database[0])]
#train_data[0][1].append(database[2][1:len(database[0])])
#print(train_data)
print(trainX[:,36])
#print(train_y)
#print(database)

In [ ]:
# Data Normalization 
# train_x_r train_y_r
train_x_r = np.zeros((len(database[0])-1,len(database)))
train_y_r = np.zeros((1,len(database)))
# Normalize the input (7 element)
for i in range(len(database[0])-1):
    if i !=0:  # negelect the output
        mintrain = min(trainX[i,:])
        maxtrain = max(trainX[i,:])
        for j in range(len(database)):
            train_x_r[i,j] = (trainX[i,j]-mintrain)/(maxtrain-mintrain)
# Normalize the output
mintrainY = min(trainY[0,:])
maxtrainY = max(trainY[0,:])
for i in range(len(database)):
    train_y_r[0,i] = (trainY[0,i]-mintrainY)/(maxtrainY-mintrainY)
    
print(trainX[:,0])
print(train_x_r[:,0])
print(trainY[:,0])
print(train_y_r[:,0])
print(min(trainY[0,:]),'\t',max(trainY[0,:]))
#for i in range(len(database[0])-1):
    #print(min(trainX[i,:]),'\t',max(trainX[i,:]))

In [ ]:
#print(train_x[:,36])
# split the training data(80%) and testing data(20%)
import random
res = random.sample(range(1, len(database)), int(len(database)*0.8))
train_x = np.zeros(((len(database[0])-1),int(len(database)*0.8)))
train_y = np.zeros((1,int(len(database)*0.8)))
#print(train_x.shape[0])

n=0
for i in res:
    train_x[:,n]=train_x_r[:,i]
    train_y[0,n]=train_y_r[0,i]
    n=n+1
#create the matrix to find 20% number of data
test_index=[]
for j in range(0,len(database)):
    n=0
    for k in res:
        if j-k !=0:
            n=n+1
    if n != len(res)-1:
        test_index.append(j)

test_x = np.zeros(((len(database[0])-1),len(test_index)))
test_y = np.zeros((1,len(test_index)))

n=0
for i in test_index:
    test_x[:,n] = train_x_r[:,i]
    test_y[0,n] = train_y_r[0,i]
    n=n+1
    
print(res)
print(train_x[:,10])
print(test_x[:,10])
#print(len(res))
#print(len(test_index))
print(".......................................")
print(".......................................")
#print(test_x[:,20])
print(test_index)
print(train_y[:,10])
print(test_y[:,10])

In [ ]:
#print(test_x[0,:])

In [ ]:
# The BPNN structure
# The initialize parameters
def NL_initialize_parameters(layer_dims):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
    
                    W1 -- weight matrix of shape (Layer_dims[W1], Layer_dims[W0])
                    b1 -- bias vector of shape (Layer_dims[W1], 1)
                    W2 -- (Layer_dims[W2], Layer_dims[W1])
                    b1 -- (Layer_dims[W2], 1)
                    
                    So we can get these rule: ("L" denotes the number of layer.)
                    Shape of "WL" -- (Layer_dims[L], Layer_dims[L-1])
                    Shape of "bL" -- (Layer_dims[L], 1)
    """
    np.random.seed(1)
    parameters = {}
    Layers = len(layer_dims)
    
    for l in range(1, Layers):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    
    return parameters
# The Forward pass
def NL_forwardpass(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """
    
    caches = []
    A = X
    L = len(parameters) // 2  
    linear_cache = {}
    activation_cache = {}
    
    for l in range(1, L):
        linear_cache = {}
        activation_cache = {}
        A_prev = A
        W = parameters['W' + str(l)]
        b = parameters['b' + str(l)]
        
        # Linear calculation
        linear_cache["A_prev" + str(l)] = A_prev
        linear_cache["W" + str(l)] = W
        linear_cache["b" + str(l)] = b
        Z = W.dot(A_prev) + b
        
        # ReLU calculation: ReLU
        A = np.maximum(0, Z)
        activation_cache["Z" + str(l)] = Z
        cache = (linear_cache, activation_cache)
        caches.append(cache)
    
    ## The final layer.
    linear_cache = {}
    activation_cache = {}
    W = parameters['W' + str(L)]
    b = parameters['b' + str(L)]
    
    # Linear calculation
    # 
    linear_cache["A_prev" + str(L)] = A
    linear_cache["W" + str(L)] = W
    linear_cache["b" + str(L)] = b
    ZL = W.dot(A) + b
    
    # Activative calculation: Sigmoid
    AL = 1/(1+np.exp(-ZL))
    activation_cache["Z" + str(L)] = ZL
    
    
    cache = (linear_cache, activation_cache)
    caches.append(cache)
    
    
    assert(AL.shape == (1,X.shape[1]))
    
    return AL, caches
# Calculate the cost, In other words, calculate the error function
def compute_cost(AL, Y):
    m = Y.shape[1]   # number of class
    # Logistic regression
    #cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    #cost = np.squeeze(cost)  # makes sure cost is the dimension we expect.
    #return cost
    return 0.5*np.sum((AL-Y)**2)
# The backward pass 
def NL_backwardpass(AL, X, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (there are (L-1) or them, indexes from 0 to L-2)
                the cache of linear_activation_forward() with "sigmoid" (there is one, index L-1)
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    ## Initializing the backpropagation
    ## Computes the gradient of AL. (AL means the y-hat of model.)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    ## L-th layer gradients.
    ## (Sigmoid -> Linear)
    current_cache = caches[L-1]   # The index of caches is in the range of 0 to L-1.
    linear_cache, activation_cache = current_cache
    
    # dZL (Sigmoid backward)
    s = 1/(1+np.exp( -activation_cache["Z" + str(L)] ))
    dZL = dAL * s * (1-s)
    
    # dA_prev (Linear backward)
    A_prev = linear_cache["A_prev" + str(L)]
    W = linear_cache["W" + str(L)]
    b = linear_cache["b" + str(L)]
    m = A_prev.shape[1]
    dW = 1./m * np.dot(dZL, A_prev.T)
    db = 1./m * np.sum(dZL, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZL)

    ## Save grads.(The L-th layer)
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = dA_prev, dW, db
    
    ## The value of l is decreased from L-1 to 1.
    for l in reversed(range(1, L)):
        ## l-th layer gradients.
        ## (ReLU -> Linear), Example: caches[2] contains: A_prev3, W3, b3
        current_cache = caches[l-1]
        linear_cache, activation_cache = current_cache
    
        # dZ (ReLU backward)
        Z = activation_cache["Z" + str(l)]
        dZ = np.array(grads["dA" + str(l)], copy=True)
        dZ[Z<=0] = 0
        assert (dZ.shape == Z.shape)  # check shape
        
        # dA (Linear backward)
        A_prev = linear_cache["A_prev" + str(l)]
        W = linear_cache["W" + str(l)]
        b = linear_cache["b" + str(l)]
        m = A_prev.shape[1]
        dW = 1./m * np.dot(dZ,A_prev.T)
        db = 1./m * np.sum(dZ, axis = 1, keepdims = True)
        dA_prev = np.dot(W.T,dZ)
        assert (dA_prev.shape == A_prev.shape)  # check shape
        assert (dW.shape == W.shape)
        assert (db.shape == b.shape)
        
        ## Save grads.(the l-th layer)
        grads["dA" + str(l-1)], grads["dW" + str(l)], grads["db" + str(l)] = dA_prev, dW, db
    
    return grads
# Update parameters
def update_parameters(parameters, grads, learning_rate = 1.2):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network
    
    # Update rule for each parameter. Use a for loop.
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    
    return parameters



In [ ]:
# The main function
"""initializing the parameter -> forward pass -> compute cost -> backward pass -> update parameter
    repeat the aformation step until the end of iteration"""
def NL_nn_model(X, Y, layers_dims, num_iterations = 5000, learning_rate=0.08, print_cost = False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    layers_dims -- size of all the hidden layers
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    costs = []
    np.random.seed(1)
    
    # Initialize W, b
    parameters = NL_initialize_parameters(layers_dims)
    
    for i in range(0, num_iterations):
        # Forward pass
        AL, caches = NL_forwardpass(X, parameters)
        # Compute cost
        cost = compute_cost(AL, Y)

        # Backward pass
        grads = NL_backwardpass(AL, X, Y, caches)
        
        # Update parameters
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the cost so far every 500 iterations
        if i % 500 == 0:
            costs.append(cost)
            if print_cost:
                print("Cost after iteration {}: {}".format(i, cost))
                #print(AL.shape)
    #print(AL)
    # Append the cost of final layer into "costs".
    costs.append(cost)
    plt.figure(num=1, figsize=(8,5))
    plt.semilogy(costs)
    plt.xlabel("Iterations")
    plt.ylabel("Cost")
    plt.title("Learning Rate = " + str(learning_rate))
    plt.show()

    return parameters
"""
    the def function cannot return AL and parameter because the parameters will change formation from
    dictionary to index when return them at the same time
"""
# Predict
def predict(X, y, parameters):
    """
    This function is used to predict the results of a  L-layer neural network.
    
    Arguments:
    X -- data set of examples you would like to label
    parameters -- parameters of the trained model
    
    Returns:
    p -- predictions for the given dataset X
    """
    
    m = X.shape[1]
    n = len(parameters) // 2 # number of layers in the neural network
    p = np.zeros((5,m))
    
    # Forward propagation
    probas, caches = NL_forwardpass(X, parameters)  
    """
    # probas is the output of NN and its shape is (5,95)
    # convert probas to 0/1 predictions
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.25:
            p[0,i] = 100
        if probas[1,i] >0.25:
            p[1,i] = 200
        if probas[2,i] > 0.25:
            p[2,i] = 300
        if probas[3,i] >0.25:
            p[3,i] = 400
        if probas[4,i] > 0.25:
            p[4,i] = 500
    """    
    #print results
    #print ("predictions: " + str(p))
    #print ("true labels: " + str(y))
    #print("Accuracy: "  + str(1-(0.5*np.sum((probas-y)**2))))
    print("Accuracy: "+str(abs(probas-y)/probas))
        
    return probas

In [ ]:
# the procedure of traning
#train_x = train_data[1]
#train_y = train_data[0]
layers_dims = [len(database[0])-1,10,3,1] # 4-Layer model
parameters = NL_nn_model(train_x, train_y, layers_dims, num_iterations=60000000,learning_rate=0.00005, print_cost=True)
# calculate the accuracy of trainning
#print("Training accuracy:")
#predictions_train = predict(train_x, train_y, parameters)

In [ ]:
print("Training accuracy:")
predictions_train = predict(train_x, train_y, parameters)
#print(train_x[:,1].shape[1])
#print(train_y.shape[1])
#print(train_x[:,1])
#print(train_x[:,1].shape)

In [ ]:
#print(predictions_train)

In [ ]:
choose_index =14
test = np.zeros((len(database[0])-1,1))
test[:,0] = test_x[:,choose_index]
answear = test_y[:,choose_index]
kk = predict(test, test_y[:,choose_index], parameters)
print("test index= "+str(test_index[choose_index]),'\t',"it can go back to check the database index")
print("Testing input:"+ str(test),"\t","it is a normalize input")
print("//////////////////the real input\\\\\\\\\\\\\\\\\\")
print("the original data = ",trainX[:,test_index[choose_index]],'\t',trainY[:,test_index[choose_index]])
print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<--------------->>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print("Prediction: "+str(kk),"The Answear:"+str(answear),"\t","it is normalize answear")
print("//////////////////the real answear\\\\\\\\\\\\\\\\\\")
print("Prediction: "+str(500*kk),"The Answear:"+str(500*answear),"\t","it is normalize answear")

#print(test_y[:,1choose_index])
#print(kk)

In [ ]:
#validation
print("test index= ",test_index[15],'\t')
print("the original data = ",trainX[:,test_index[15]],'\t',trainY[:,test_index[15]])
#print("the normalize data = ",trainX[:,test_index[10]],'\t',trainY[:,test_index[10]])


In [ ]:
print(len(test_index))

In [ ]:
type(parameters)

In [ ]:
import hdfdict as hdf
# save the "parameters" due to "parameters" is a dictionary
fname  = 'parameters.h5'
hdf.dump(parameters,fname)

In [ ]:
para = hdf.load('parameters.h5')
s =dict(para) # make it open